# Tapis v3 Gateways 2021 Hands-on

In this notebook, you will use Tapis v3 to create two systems and one application that will be used to run
an image classification job on both a VM and an HPC cluster.

To execute each `In[#]` cell, you can click inside the cell and press `Shift + Enter`

## Enter training account information

To get things started, please run the following and enter the training account information provided to you:

In [ ]:
import getpass

tenant = 'tacc'
base_url = 'https://' + tenant + '.tapis.io'

username = input('Username: ')
password = getpass.getpass(prompt='Password: ', stream=None)
host = input('Host: ')

## Authenticate and initialize Tapis v3 client

Using this information, you can now use `tapipy` to authenticate in the tenant and initialize the
Tapis v3 client. You should see your token information displayed.

In [ ]:
from tapipy.tapis import Tapis
#Create python Tapis client for user
client = Tapis(base_url= base_url, username=username, password=password)
# *** Tapis v3: Call to Tokens API
client.get_tokens()
# Print Tapis v3 token
client.access_token

## Systems

In this section we create two Tapis systems, one for a training VM host and one for the stampede2 HPC cluster.

Note that although it is possible, we have not provided any login credentials in the system definitions.
Well-crafted system definitions are likely to be copied and re-used, so, for security reasons, it is recommended that
login credentials be registered using separate API calls as discussed below.

### Create a system for the VM host

In [ ]:
user_id = username
system_id_vm = "gateways21-vm-" + user_id

# Create the system definition
exec_system_vm = {
  "id": system_id_vm,
  "description": "System for testing jobs on a VM for gateways2021 tutorial",
  "systemType": "LINUX",
  "host": host,
  "defaultAuthnMethod": "PASSWORD",
  "port": 22,
  "rootDir": "/home/"+user_id,
  "canExec": True,
  "jobRuntimes": [ { "runtimeType": "SINGULARITY" } ],
  "jobWorkingDir": "workdir",
  "jobIsBatch": True,
  "batchScheduler": "SLURM",
  "batchDefaultLogicalQueue": "tapisNormal",
  "batchLogicalQueues": [
    {
      "name": "tapisNormal",
      "hpcQueueName": "debug",
      "maxJobs": 50,
      "maxJobsPerUser": 10,
      "minNodeCount": 1,
      "maxNodeCount": 16,
      "minCoresPerNode": 1,
      "maxCoresPerNode": 68,
      "minMemoryMB": 1,
      "maxMemoryMB": 16384,
      "minMinutes": 1,
      "maxMinutes": 60
    }
  ]
}

# Use the client to create the system in Tapis
print("****************************************************")
print("Create system: " + system_id_vm)
print("****************************************************")
client.systems.createSystem(**exec_system_vm)

In [ ]:
# List all systems available to you
print("****************************************************")
print("List all systems")
print("****************************************************")
client.systems.getSystems()

In [ ]:
# Get details for the system you created
print("****************************************************")
print("Fetch system: " + system_id_vm)
print("****************************************************")
client.systems.getSystem(systemId=system_id_vm)

### Register Credentials for the VM system

After creating the system, you will need to register credentials for your username. These will be used by Tapis to
access the host. Various authentication methods can be used to access a system, such as PASSWORD and PKI_KEYS. For the
VM a password is used.

In [ ]:
password_vm = password
# Register credentials
client.systems.createUserCredential(systemId=system_id_vm, userName=user_id, password=password_vm)

Now you can use the client to list files on the system. This will confirm that the credentials are valid.

In [ ]:
# List files at the rootDir for the system
client.files.listFiles(systemId=system_id_vm, path="/")

### Create a system for the HPC cluster

With just a few changes to the system definition you can create a second system that can be used to run the
same application on an HPC cluster. Note the minimal changes:

* **id** - A unique id is required
* **host** - This is now the main hostname for the HPC cluster
* **rootDir** - Using the root directory of the host gives us flexibility in setting **jobWorkingDir**.
  Note that you still need LINUX permissions.
* **jobWorkingDir** - Now determined dynamically using the Tapis v3 function HOST_EVAL()
* **batchLogicalQueue.hpcQueueName** - A different underlying queue name is used for the HPC cluster.

In [ ]:
user_id = username
system_id_hpc = "gateways21-hpc-" + user_id

# Create the system definition
exec_system_hpc = {
  "id": system_id_hpc,
  "description": "System for testing jobs on an HPC cluster for gateways2021 tutorial",
  "systemType": "LINUX",
  "host": "stampede2.tacc.utexas.edu",
  "defaultAuthnMethod": "PASSWORD",
  "port": 22,
  "rootDir": "/",
  "canExec": True,
  "jobRuntimes": [ { "runtimeType": "SINGULARITY" } ],
  "jobWorkingDir": "HOST_EVAL($WORK2)",
  "jobIsBatch": True,
  "batchScheduler": "SLURM",
  "batchSchedulerProfile": "tacc",
  "batchDefaultLogicalQueue": "tapisNormal",
  "batchLogicalQueues": [
    {
      "name": "tapisNormal",
      "hpcQueueName": "normal",
      "maxJobs": 50,
      "maxJobsPerUser": 10,
      "minNodeCount": 1,
      "maxNodeCount": 16,
      "minCoresPerNode": 1,
      "maxCoresPerNode": 68,
      "minMemoryMB": 1,
      "maxMemoryMB": 16384,
      "minMinutes": 1,
      "maxMinutes": 60
    }
  ]
}

# Use the client to create the system in Tapis
print("****************************************************")
print("Create system: " + system_id_hpc)
print("****************************************************")
client.systems.createSystem(**exec_system_hpc)

In [ ]:
# List all systems available to you
print("****************************************************")
print("List all systems")
print("****************************************************")
client.systems.getSystems()

In [ ]:
# Get details for the system you created
print("****************************************************")
print("Fetch system: " + system_id_hpc)
print("****************************************************")
client.systems.getSystem(systemId=system_id_hpc)

### Register Credentials for the HPC system

As before, now you will need to register credentials for your username. These will be used by Tapis to
access the host.

In [ ]:
password_hpc = password
# Register credentials
client.systems.createUserCredential(systemId=system_id_hpc, userName=user_id, password=password_hpc)

Now you can use the client to list files on the system. This will confirm that the credentials are valid.

In [ ]:
# List files at the rootDir for the system
client.files.listFiles(systemId=system_id_hpc, path="/")

## Application

In order to run a job on a system you will need to create a Tapis application.

### Create an application that can be run on the VM host or the HPC cluster

In [ ]:
user_id = username
app_id = "gateways21-img-classify-" + user_id

# Create the application definition
app_def = {
  "id": app_id,
  "version": "0.0.1",
  "description": "Image classifier run using Singularity in batch mode",
  "appType": "BATCH",
  "runtime": "SINGULARITY",
  "runtimeOptions": ["SINGULARITY_RUN"],
  "containerImage": "docker://tapis/img-classify:0.1",
  "jobAttributes": {
    "parameterSet": {
      "archiveFilter": { "includeLaunchFiles": False }
    },
    "memoryMB": 1,
    "nodeCount": 1,
    "coresPerNode": 1,
    "maxMinutes": 10
  }
}

# Use the client to create the application in Tapis
print("****************************************************")
print("Create application: " + app_id)
print("****************************************************")
client.apps.createAppVersion(**app_def)

In [ ]:
# List all applications available to you
print("****************************************************")
print("List all applications")
print("****************************************************")
client.apps.getApps()

In [ ]:
# Get details for the application you created
print("****************************************************")
print("Fetch application: " + app_id)
print("****************************************************")
client.apps.getAppLatestVersion(appId=app_id)

## Jobs

In [ ]:
# Run Image classifier app on the Virtual Machine
# In the arg pass a url of the image you would like to classify
pa = {
 "parameterSet": {
      "appArgs": [
          {"arg": "--image_file", "meta": { "name": "arg1", "required": True}},
          {"arg": "'https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12231410/Labrador-Retriever-On-White-01.jpg'",
           "meta": {"name": "arg2", "required": True}
          }
      ]
 }
}
# Submit a job
job_output=client.jobs.submitJob(name='img-classifier-job-vm',description='image classifier',appId=app_id,execSystemId=system_id_vm,appVersion= '0.0.1',
  **pa)

In [ ]:
print("****************************************************")
print("Job Submitted: " + app_id)
print("****************************************************")
print(job_output)

In [ ]:
# Get job uuid from the job submission response
print("****************************************************")
job_uuid=job_output.uuid
print("Job UUID: " + job_uuid)
print("****************************************************")

In [ ]:
# Check the status of the job
print("****************************************************")
job_uuid=job_output.uuid
print(client.jobs.getJobStatus(jobUuid=job_uuid))
print("****************************************************")

In [ ]:
# To cancel a running job
client.jobs.cancelJob(jobUuid='0777ed62-b746-4b2d-82cd-9c0e616e91df-007')

In [ ]:
# Download output of the job
print("Job Output file:")

print("****************************************************")
job_uuid=job_output.uuid
print(client.jobs.getJobOutputDownload(jobUuid=job_uuid,outputPath='tapisjob.out'))
print("****************************************************")

Run Job on HPC
We can use the same Tapis application, that we registered earlier to run on a HPC machine
We have already registered execution system on Stampede2

In [ ]:
# Run Image classifier app on the Virtual Machine
# In the arg pass a url of the image you would like to classify
pa = {
 "parameterSet": {
      "appArgs": [
          {"arg": "--image_file"},
          {"arg": "'https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12231410/Labrador-Retriever-On-White-01.jpg'"}
      ],
      "schedulerOptions": [
        {"arg": "--tapis-profile tacc"}
      ]
 }
}
# Submit a job
job_output=client.jobs.submitJob(name='img-classifier-job-vm',description='image classifier',appId=app_id,execSystemId=system_id_hpc,appVersion= '0.0.1',
  **pa)

In [ ]:
print("****************************************************")
print("Job Submitted: " + app_id)
print("****************************************************")
print(job_output)

In [ ]:
# Check the status of the job
print("****************************************************")
job_uuid=job_output.uuid
print(client.jobs.getJobStatus(jobUuid=job_uuid))
print("****************************************************")

In [ ]:
# Download output of the job
print("Job Output file:")

print("****************************************************")
job_uuid=job_output.uuid
print(client.jobs.getJobOutputDownload(jobUuid=job_uuid,outputPath='tapisjob.out'))
print("****************************************************")